 Graniczna Analiza Danych — Raport

## Autorzy
- Daniel Zdancewicz [145317]
- Alex Terentowicz [145419]


## Opis zbiorów danych

Zbiór danych zawierający informacje o lotniskach

[Wejścia](./resources/datasets/inputs.csv):
- i1: roczna przepustowość terminalu zdefiniowana jako przepływ pasażerów, który port lotniczy może obsłużyć bez poważnych niedogodności (w milionach pasażerów rocznie)
- i2: maksymalna przepustowość zdefiniowana jako średnia liczba operacji (przylotów i/lub odlotów), które można wykonać na pasach startowych portu lotniczego (w liczbie operacji na godzinę)
- i3: przepustowość płyty postojowej lotniska definiowana jako średnia liczba samolotów, które może obsłużyć lotnisko (w liczbie samolotów na godzinę)
- i4: obszar ciążenia lotniska zdefiniowany jako liczba mieszkańców mieszkających w promieniu 100 km od lotniska (w mln mieszkańców)

[Wyjścia](./resources/datasets/outputs.csv):
- o1: ruch pasażerski mierzony całkowitą liczbą pasażerów obsłużonych przez port (w mln pasażerów rocznie)
- o2: liczba operacji statku powietrznego (jeden całkowity ruch to lądowanie lub start statku powietrznego; w tysiącach ruchów rocznie)


[Próbki](./resources/datasets/samples.csv):
- i1: waga wejścia i1
- i2: waga wejścia i2
- i3: waga wejścia i3
- i4: waga wejścia i4
- o1: waga wyjścia i1
- o2: waga wyjścia i2


### Odczyt danych

In [8]:
import pandas as pd


def load(path: str):
  return pd.read_csv(path, sep=';')


inputs_ds = load('./resources/datasets/inputs.csv')
outputs_ds = load('./resources/datasets/outputs.csv')
samples_ds = load('./resources/datasets/samples.csv')

In [9]:
inputs_ds.head()

,Unnamed: 0,i1,i2,i3,i4
0,WAW,10.5,36,129.4,7.0
1,KRK,3.1,19,31.6,7.9
2,KAT,3.6,32,57.4,10.5
3,WRO,1.5,12,18.0,3.0
4,POZ,1.5,10,24.0,4.0


In [20]:
outputs_ds.head()

,Unnamed: 0,o1,o2
0,WAW,9.5,129.7
1,KRK,2.9,31.3
2,KAT,2.4,21.1
3,WRO,1.5,18.8
4,POZ,1.3,16.2


In [11]:
samples_ds.head()

,sample,i1,i2,i3,i4,o1,o2
0,1,0.096443,0.059350,0.065224,0.778983,0.620824,0.379176
1,2,0.032029,0.304959,0.427426,0.235586,0.513536,0.486464
2,3,0.266256,0.246940,0.217498,0.269306,0.247490,0.752510
3,4,0.159062,0.635512,0.202159,0.003267,0.223838,0.776162
4,5,0.096716,0.809025,0.026741,0.067518,0.136140,0.863860


## Efektywność

Do wyznaczenia efektywności został utworzony model analizy danych **CCR** zorientowany na nakłady ( wejścia ).

\begin{align*}
\max & \sum_{n=1}^{N}\mu_n\cdot y_{no} & \\
s.t. & \sum\limits_{m=1}^{M}\nu_m\cdot x_{mo}=1  &  \\
 & \sum\limits_{n=1}^{N}\mu_n\cdot y_{nk}\leq\sum\limits_{m=1}^{M}\nu_n\cdot x_{mk}, & k=1,2,...,K \\
 & \mu_n\nu_m\geq0, & m=1,2,...,M\;, n=1,2,...,N \\
\end{align*}



In [146]:
import pulp
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, LpMinimize


def create_model(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  N, M, K = map(len, [outputs_df.columns[1:], inputs_df.columns[1:], inputs_df])
  o = alternative_nr

  model = LpProblem("CCR-input", LpMaximize)

  input_weights = LpVariable.dicts("input_weight", range(M), 0, None, cat='Continuous')
  output_weights = LpVariable.dicts("output_weight", range(N), 0, None, cat='Continuous')

  inputs = inputs_df.iloc[:, 1:].values.T
  outputs = outputs_df.iloc[:, 1:].values.T

  model.setObjective(sum(
    output_weights[n] * outputs[n][o] for n in range(N)
  ))

  model.addConstraint(sum(
    input_weights[m] * inputs[m][o] for m in range(M)
  ) == 1)

  for k in range(K):
    model.addConstraint(
      sum(output_weights[n] * outputs[n][k] for n in range(N))
      <=
      sum(input_weights[m] * inputs[m][k] for m in range(M))
    )

  return model


def find_efficiency(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  model = create_model(alternative_nr, inputs_ds, outputs_ds)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.objective.value()


for k in range(len(inputs_ds)):
  print(
    f'Efficiency for alternative {inputs_ds[inputs_ds.columns[0]][k]} {k + 1:0>2}: {find_efficiency(k, inputs_ds, outputs_ds) * 100:.2f}%')


Efficiency for alternative WAW 01: 100.00%
Efficiency for alternative KRK 02: 100.00%
Efficiency for alternative KAT 03: 59.12%
Efficiency for alternative WRO 04: 100.00%
Efficiency for alternative POZ 05: 79.98%
Efficiency for alternative LCJ 06: 30.00%
Efficiency for alternative GDN 07: 100.00%
Efficiency for alternative SZZ 08: 27.08%
Efficiency for alternative BZG 09: 100.00%
Efficiency for alternative RZE 10: 40.92%
Efficiency for alternative IEG 11: 25.85%


Lotniska efektywne to: Warszawa, Kraków, Wrocław, Gdynia, Bydgoszcz.
LOtniska nieefektywne to: Katowice, Poznań, Łódź, Szczecin, Rzeszów, Zielona Góra.

## Hipotetyczna jednostka porównawcza oraz potrzebne poprawki

Dla każdej z odnalezionych jednostek nieefektywnych odnaleziono hipotetyczną jednostkę porównawczą oraz poprawki potrzebne do osiągnięcia efektywności.


\begin{align*}
\min & \theta & \\
s.t. & \sum\limits_{k=1}^{K}\lambda_k\cdot x_{nk}\leq\theta\cdot x_{no}  & n=1,2,...,N  \\
 & \sum\limits_{k=1}^{K}\lambda_k\cdot y_{nk}\geq y_{mo}, & m=1,2,...,M \\
 & \theta\geq0, &  \\
 & \lambda_k\geq0, & k=1,2,...,K \\
\end{align*}


In [178]:
def create_model(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  N, M, K = map(len, [outputs_df.columns[1:], inputs_df.columns[1:], inputs_df])
  o = alternative_nr
  inputs = inputs_df.iloc[:, 1:].values.T
  outputs = outputs_df.iloc[:, 1:].values.T

  model = LpProblem("CCR-input", LpMinimize)

  efficiency = LpVariable("efficiency", 0, None, cat='Continuous')
  lambdas = LpVariable.dicts("lambda", range(K), 0, None, cat='Continuous')

  model.setObjective(efficiency)

  for m in range(M): model.addConstraint(
    sum(lambdas[k] * inputs[m][k] for k in range(K))
    <=
    efficiency * inputs[m][o]
  )

  for n in range(N): model.addConstraint(
    sum(lambdas[k] * outputs[n][k] for k in range(K))
    >=
    outputs[n][o]
  )

  return model


def find_(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  N, M, K = map(len, [outputs_df.columns[1:], inputs_df.columns[1:], inputs_df])
  o = alternative_nr
  inputs = inputs_df.iloc[:, 1:].values.T
  outputs = outputs_df.iloc[:, 1:].values.T

  model = create_model(alternative_nr, inputs_ds, outputs_ds)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  vars = model.variablesDict()

  input_hcus = [
    sum(vars[f'lambda_{k}'].value() * inputs[m][k] for k in range(K))
    for m in range(M)
  ]
  output_hcus = [
    sum(vars[f'lambda_{k}'].value() * outputs[n][k] for k in range(K))
    for n in range(N)
  ]

  input_differences = [
    inputs[m][o] - input_hcus[m]
    for m in range(M)
  ]
  output_differences = [
    output_hcus[n] - outputs[n][o]
    for n in range(N)
  ]

  return pd.DataFrame(
    data=[
      input_differences,
      output_differences
    ],
    index=[
      'input_differences',
      'output_differences'
    ],
  )


for k in range(len(inputs_ds)):
  print(f"differences for alternative {inputs_ds[inputs_ds.columns[0]][k]} {k + 1}:\n{find_(k, inputs_ds, outputs_ds)}")


differences for alternative WAW 1:
                      0    1    2    3
input_differences   0.0  0.0  0.0  0.0
output_differences  0.0  0.0  NaN  NaN
differences for alternative KRK 2:
                      0    1    2    3
input_differences   0.0  0.0  0.0  0.0
output_differences  0.0  0.0  NaN  NaN
differences for alternative KAT 3:
                               0          1          2         3
input_differences   1.471647e+00  13.081308  23.464596  6.103519
output_differences  7.400000e-09   8.569860        NaN       NaN
differences for alternative WRO 4:
                      0    1    2    3
input_differences   0.0  0.0  0.0  0.0
output_differences  0.0  0.0  NaN  NaN
differences for alternative POZ 5:
                               0             1         2         3
input_differences   3.002985e-01  2.001990e+00  4.804776  2.071571
output_differences  2.700000e-09  3.310000e-08       NaN       NaN
differences for alternative LCJ 6:
                           0             1 

## Superefektywność

Do wyznaczenia superefektywności należy z modelu zdjąć ograniczenie efektywności dla kolejnych kryteriów.

\begin{align*}
\max & \sum_{n=1}^{N}\mu_n\cdot y_{no} & \\
s.t. & \sum\limits_{m=1}^{M}\nu_m\cdot x_{mo}=1  &  \\
 & \sum\limits_{n=1}^{N}\mu_n\cdot y_{nk}\leq\sum\limits_{m=1}^{M}\nu_n\cdot x_{mk}, & k=1,2,...,K,\;k\neq o \\
 & \mu_n\nu_m\geq0, & m=1,2,...,M\;, n=1,2,...,N \\
\end{align*}



In [135]:
import pulp
from pulp import LpVariable, LpProblem, LpMaximize


def create_model(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  N, M, K = map(len, [outputs_df.columns[1:], inputs_df.columns[1:], inputs_df])
  o = alternative_nr

  model = LpProblem("CCR-input", LpMaximize)

  input_weights = LpVariable.dicts("input_weight", range(M), 0, None, cat='Continuous')
  output_weights = LpVariable.dicts("output_weight", range(N), 0, None, cat='Continuous')

  inputs = inputs_df.iloc[:, 1:].values.T
  outputs = outputs_df.iloc[:, 1:].values.T

  model.setObjective(sum(
    output_weights[n] * outputs[n][o] for n in range(N)
  ))

  model.addConstraint(sum(
    input_weights[m] * inputs[m][o] for m in range(M)
  ) == 1)

  for k in filter(lambda k: k != o, range(K)):
    model.addConstraint(
      sum(output_weights[n] * outputs[n][k] for n in range(N))
      <=
      sum(input_weights[m] * inputs[m][k] for m in range(M))
    )

  return model


def find_efficiency(alternative_nr: int, inputs_df: pd.DataFrame, outputs_df: pd.DataFrame):
  model = create_model(alternative_nr, inputs_ds, outputs_ds)
  model.solve(pulp.PULP_CBC_CMD(msg=False))
  return model.objective.value()


for k in range(len(inputs_ds)):
  print(
    f'Super-efficiency for alternative {inputs_ds[inputs_ds.columns[0]][k]} {k + 1:0>2}: {find_efficiency(k, inputs_ds, outputs_ds) * 100:.2f}%')


Super-efficiency for alternative WAW 01: 227.79%
Super-efficiency for alternative KRK 02: 112.38%
Super-efficiency for alternative KAT 03: 59.12%
Super-efficiency for alternative WRO 04: 103.99%
Super-efficiency for alternative POZ 05: 79.98%
Super-efficiency for alternative LCJ 06: 30.00%
Super-efficiency for alternative GDN 07: 200.00%
Super-efficiency for alternative SZZ 08: 27.08%
Super-efficiency for alternative BZG 09: 174.59%
Super-efficiency for alternative RZE 10: 40.92%
Super-efficiency for alternative IEG 11: 25.85%


## Efektywność krzyżowa

## Rozkład efektywności

## Ranking jednostek
